# Data Cleaning Listado de Listado de Beneficiarios 2019

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_19 -> Dataset con el listado de TODOS los beneficiarios de 2019.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2019

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_19 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/fertilizantes_2019.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_19 = beneficiarios_19[~((beneficiarios_19['ENTIDAD'] == 'NACIONAL') & (beneficiarios_19['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_19['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_19.shape

(275026, 16)

In [6]:
beneficiarios_19['ENTIDAD'].unique()

array(['GUERRERO', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [8]:
beneficiarios_19.dropna(inplace=True)

In [10]:
# Obtenemos las localidades únicas en el dataset.
Localidades_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_19 = Localidades_19.drop_duplicates()


In [11]:
Localidades_19['ENTIDAD_c_benef'] = Localidades_19['ENTIDAD'].apply(clean_text)
Localidades_19['MUNICIPIO_c_benef'] = Localidades_19['MUNICIPIO'].apply(clean_text)
Localidades_19['LOCALIDAD_c_benef'] = Localidades_19['LOCALIDAD'].apply(clean_text)

In [12]:
# Creamos las dos keys de beneficiarios 2019
Localidades_19['KEY_benef_mun'] = Localidades_19['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_19[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_19['KEY_benef_loc'] = Localidades_19['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_19[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_19['LOCALIDAD_c_benef'].astype(str)

In [13]:
Localidades_19.shape

(4576, 8)

In [14]:
# Obtenemos las localidades únicas en el dataset.
Municipios_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO']]
Municipios_19 = Municipios_19.drop_duplicates()

In [15]:
# Estandarizamos la limpieza de los datos
Municipios_19['ENTIDAD_c_benef'] = Municipios_19['ENTIDAD'].apply(clean_text)
Municipios_19['MUNICIPIO_c_benef'] = Municipios_19['MUNICIPIO'].apply(clean_text)

In [16]:
# Creamos las dos keys de beneficiarios 
Municipios_19['KEY_benef_mun'] = Municipios_19['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_19[
    'MUNICIPIO_c_benef'].astype(str)

In [17]:
Municipios_19.shape

(81, 5)

In [18]:
Municipios_19.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2019

Obtener listado único de municipios y listado único de localidades de inegi de 2019 con sus claves.

In [19]:
path_dataset_inegi_2019 = '../../data/inegi/dataset_inegi_clean_2019.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2019 = pd.read_csv(path_dataset_inegi_2019)

### 1.2.1 INEGI 2019 Municipios únicos para cada año.

In [20]:
dataset_inegi_2019['KEY_inegi_municipio'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2019['KEY_inegi_localidad'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2019['Localidad_c_inegi'].astype(str)

In [21]:
INEGI_UNIQUEMUN_2019 = dataset_inegi_2019.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2019 = INEGI_UNIQUEMUN_2019.drop_duplicates()

In [22]:
INEGI_UNIQUEMUN_2019.shape

(2465, 7)

In [23]:
INEGI_UNIQUEMUN_2019.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

### 1.2.1 INEGI 2019 Localidades únicas para cada año.

In [24]:
INEGI_UNIQUELOC_2019 = dataset_inegi_2019

INEGI_UNIQUELOC_2019 = INEGI_UNIQUELOC_2019.drop_duplicates()

In [25]:
INEGI_UNIQUELOC_2019.shape

(299570, 11)

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2019, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_19 y el dataset de INEGI correspondiente.

In [26]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [27]:
diccionario_MUN_19 = fuzzy_merge_benef2019_2022(Municipios_19, INEGI_UNIQUEMUN_2019, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_19.shape

(81, 14)

In [28]:
diccionario_MUN_19.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [29]:
diccionario_MUN_19.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19.csv', index=False)

In [30]:
diccionario_MUN_19_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19_simple.csv')

In [31]:
nan_rows = beneficiarios_19[beneficiarios_19.isna().any(axis=1)]

In [32]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [33]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_19[beneficiarios_19['ENTIDAD'] == 'NACIONAL']

In [34]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [35]:
beneficiarios_19.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [36]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_19['ESTADO_Clean'] = beneficiarios_19['ENTIDAD'].apply(clean_text)
beneficiarios_19['MUNICIPIO_Clean'] = beneficiarios_19['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_19['Estado-mun-KEY'] = beneficiarios_19['ESTADO_Clean'].astype(str) + '-' + beneficiarios_19[
    'MUNICIPIO_Clean'].astype(str)

In [37]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_19, diccionario_MUN_19_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [38]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [39]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2019, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [40]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi
0,TORRES ROMERO SANTANA,CENTRO PAÍS,GUERRERO,AZOYÚ,LOS CHEGÜES,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,azoyu,guerrero-azoyu,guerrero-azoyu,guerrero-azoyu,12,Guerrero,13,Azoyú,guerrero,azoyu
1,GOMEZ PADILLA DEMETRIA,CENTRO PAÍS,GUERRERO,TECOANAPA,MECATEPEC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa,guerrero,tecoanapa
2,GUERRERO MELCHOR ADRIAN,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TLAJOCOTLA (LA HACIENDA),Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,teloloapan,guerrero-teloloapan,guerrero-teloloapan,guerrero-teloloapan,12,Guerrero,58,Teloloapan,guerrero,teloloapan
3,ROMERO GONZALEZ CELERINA,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TELOLOAPAN,Medio,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,teloloapan,guerrero-teloloapan,guerrero-teloloapan,guerrero-teloloapan,12,Guerrero,58,Teloloapan,guerrero,teloloapan
4,GOMEZ CERON MA.MIREYA,CENTRO PAÍS,GUERRERO,TECOANAPA,LOS SAUCES,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa,guerrero,tecoanapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275018,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón,guerrero,taxco de alarcon
275019,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón,guerrero,taxco de alarcon
275020,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,cocula,guerrero-cocula,guerrero-cocula,guerrero-cocula,12,Guerrero,17,Cocula,guerrero,cocula
275021,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,atlixtac,guerrero-atlixtac,guerrero-atlixtac,guerrero-atlixtac,12,Guerrero,10,Atlixtac,guerrero,atlixtac


In [41]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi'],
      dtype='object')

In [42]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio', 'Entidad_c_inegi','Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [43]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,TORRES ROMERO SANTANA,CENTRO PAÍS,GUERRERO,AZOYÚ,LOS CHEGÜES,Cultivo de maíz grano,30/06/2019,2430.55,PV2019,guerrero-azoyu,12,Guerrero,13,Azoyú
1,GOMEZ PADILLA DEMETRIA,CENTRO PAÍS,GUERRERO,TECOANAPA,MECATEPEC,Cultivo de maíz grano,02/07/2019,4383.10,PV2019,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa
2,GUERRERO MELCHOR ADRIAN,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TLAJOCOTLA (LA HACIENDA),Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-teloloapan,12,Guerrero,58,Teloloapan
3,ROMERO GONZALEZ CELERINA,CENTRO PAÍS,GUERRERO,TELOLOAPAN,TELOLOAPAN,Cultivo de maíz grano,17/07/2019,4383.10,PV2019,guerrero-teloloapan,12,Guerrero,58,Teloloapan
4,GOMEZ CERON MA.MIREYA,CENTRO PAÍS,GUERRERO,TECOANAPA,LOS SAUCES,Cultivo de maíz grano,18/07/2019,4383.10,PV2019,guerrero-tecoanapa,12,Guerrero,56,Tecoanapa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275018,CASTILLO CUEVAS AMANCIO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón
275019,RUEDA SECUNDINO ANACLETO,CENTRO PAÍS,GUERRERO,TAXCO DE ALARCÓN,PAINTLA,Cultivo de maíz grano,27/06/2019,2430.55,PV2019,guerrero-taxco de alarcon,12,Guerrero,55,Taxco de Alarcón
275020,ARMENTA VILLEGAS LUIS,CENTRO PAÍS,GUERRERO,COCULA,COCULA,Cultivo de maíz grano,06/07/2019,2430.55,PV2019,guerrero-cocula,12,Guerrero,17,Cocula
275021,BELLO JIMENEZ GREGORIA,CENTRO PAÍS,GUERRERO,ATLIXTAC,XALPITZÁHUAC,Cultivo de maíz grano,15/07/2019,6574.65,PV2019,guerrero-atlixtac,12,Guerrero,10,Atlixtac


In [44]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [45]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2019.csv', index=False)

In [46]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

# 3. Join por entidad (Divide y Vencerás)

### 3.1.1 Join de localidades - PUEBLA

In [47]:
Localidades_19_PUEBLA = listado_beneficiarios_parte_II[listado_beneficiarios_parte_II["ENTIDAD"] == "PUEBLA"]

dataset_inegi_uniqueloc_PUEBLA = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Puebla']

In [48]:
Localidades_19_PUEBLA = fuzzy_merge_benef2019_2022(Localidades_19_PUEBLA, dataset_inegi_uniqueloc_PUEBLA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_PUEBLA.shape

KeyError: 'KEY_benef_loc'

### 3.2.2 Join de localidades - CHIAPAS

In [ ]:
Localidades_19_CHIAPAS = Localidades_19[Localidades_19["ENTIDAD"] == "CHIAPAS"]

dataset_inegi_uniqueloc_CHIAPAS = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Chiapas']

In [ ]:
Localidades_19_CHIAPAS = fuzzy_merge_benef2019_2022(Localidades_19_CHIAPAS, dataset_inegi_uniqueloc_CHIAPAS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_CHIAPAS.shape

(0, 21)

### 3.2.3 Join de localidades - DURANGO

In [ ]:
Localidades_19_DURANGO = Localidades_19[Localidades_19["ENTIDAD"] == "DURANGO"]

dataset_inegi_uniqueloc_DURANGO = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Durango']

In [ ]:
Localidades_19_DURANGO = fuzzy_merge_benef2019_2022(Localidades_19_DURANGO, dataset_inegi_uniqueloc_DURANGO, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_DURANGO.shape

(0, 21)

### 3.2.4 Join de localidades - MORELOS

In [ ]:
Localidades_19_MORELOS = Localidades_19[Localidades_19["ENTIDAD"] == "MORELOS"]

dataset_inegi_uniqueloc_MORELOS = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Morelos']

In [ ]:
Localidades_19_MORELOS = fuzzy_merge_benef2019_2022(Localidades_19_MORELOS, dataset_inegi_uniqueloc_MORELOS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_MORELOS.shape

(0, 21)

### 3.2.5 Join de localidades - GUERRERO

In [ ]:
Localidades_19_GUERRERO = Localidades_19[Localidades_19["ENTIDAD"] == "GUERRERO"]

dataset_inegi_uniqueloc_GUERRERO = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Guerrero']

In [ ]:
Localidades_19_GUERRERO = fuzzy_merge_benef2019_2022(Localidades_19_GUERRERO, dataset_inegi_uniqueloc_GUERRERO, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_GUERRERO.shape

### 3.2.6 Join de localidades - NAYARIT

In [ ]:
Localidades_19_NAYARIT = Localidades_19[Localidades_19["ENTIDAD"] == "NAYARIT"]

dataset_inegi_uniqueloc_NAYARIT = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Nayarit']

In [ ]:
Localidades_19_NAYARIT = fuzzy_merge_benef2019_2022(Localidades_19_NAYARIT, dataset_inegi_uniqueloc_NAYARIT, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_NAYARIT.shape

(0, 21)

### 3.2.7 Join de localidades - OAXACA

In [ ]:
Localidades_19_OAXACA = Localidades_19[Localidades_19["ENTIDAD"] == "OAXACA"]

dataset_inegi_uniqueloc_OAXACA = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Oaxaca']

In [ ]:
Localidades_19_OAXACA = fuzzy_merge_benef2019_2022(Localidades_19_OAXACA, dataset_inegi_uniqueloc_OAXACA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_OAXACA.shape

(0, 21)

### 3.2.8 Join de localidades - TLAXCALA

In [ ]:
Localidades_19_TLAXCALA = Localidades_19[Localidades_19["ENTIDAD"] == "TLAXCALA"]

dataset_inegi_uniqueloc_TLAXCALA = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Tlaxcala']

In [ ]:
Localidades_19_TLAXCALA = fuzzy_merge_benef2019_2022(Localidades_19_TLAXCALA, dataset_inegi_uniqueloc_TLAXCALA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_TLAXCALA.shape

(0, 21)

### 3.2.9 Join de localidades - ZACATECAS

In [ ]:
Localidades_19_ZACATECAS = Localidades_19[Localidades_19["ENTIDAD"] == "ZACATECAS"]

dataset_inegi_uniqueloc_ZACATECAS = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Zacatecas']

In [ ]:
Localidades_19_ZACATECAS = fuzzy_merge_benef2019_2022(Localidades_19_ZACATECAS, dataset_inegi_uniqueloc_ZACATECAS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_ZACATECAS.shape

(0, 21)

In [ ]:
Localidades_19_GUERRERO.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/Localidades_19_GUERRERO.csv', index=False)